[Pytorch tuto](https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/deep_learning_60min_blitz.html)

# Pytorch란
> python 기반의 과학 연산 패키지

- numpy를 대체, GPU의 연산력을 사용
- 최대한의 유연성과 속도를 제공하는 딥러닝 연구 플랫폼

## Tensors
- numpy의 ndarray와 유사할 뿐만 아니라, GPU를 사용한 연산 가속도 지원

In [1]:
from __future__ import print_function
import torch

In [2]:
# 초기화되지 않은 5x3 행렬 생성
x = torch.Tensor(5, 3)
print(x)

tensor([[3.0811e-23, 8.5339e-43, 3.0811e-23],
        [8.5339e-43, 3.0811e-23, 8.5339e-43],
        [3.0811e-23, 8.5339e-43, 3.0806e-23],
        [8.5339e-43, 3.0806e-23, 8.5339e-43],
        [3.0810e-23, 8.5339e-43, 3.0810e-23]])


In [3]:
# 무작위로 초기화된 행렬 생성
x = torch.rand(5, 3)
print(x)

tensor([[0.3066, 0.2013, 0.5141],
        [0.0769, 0.9988, 0.5029],
        [0.1234, 0.0809, 0.8736],
        [0.3147, 0.6114, 0.5459],
        [0.5037, 0.6788, 0.6910]])


In [4]:
# 행렬 크기 구함
print(x.size())

torch.Size([5, 3])


`torch.Size`는 튜플과 같으며 모든 튜플 연산에 사용할 수 있음

## 연산
- 연산을 위한 여러가지 문법 제공

In [6]:
y = torch.rand(5, 3)
print(x+y)

tensor([[0.7308, 1.1302, 1.0226],
        [0.6929, 1.6696, 0.8362],
        [1.0237, 0.2497, 1.7533],
        [1.2015, 0.8582, 1.1531],
        [0.6830, 1.5065, 1.2289]])


In [7]:
print(torch.add(x, y))

tensor([[0.7308, 1.1302, 1.0226],
        [0.6929, 1.6696, 0.8362],
        [1.0237, 0.2497, 1.7533],
        [1.2015, 0.8582, 1.1531],
        [0.6830, 1.5065, 1.2289]])


In [9]:
result = torch.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[0.7308, 1.1302, 1.0226],
        [0.6929, 1.6696, 0.8362],
        [1.0237, 0.2497, 1.7533],
        [1.2015, 0.8582, 1.1531],
        [0.6830, 1.5065, 1.2289]])


### in-place
> tensor의 값을 변경하는 연산은 `_`를 접미사로 가짐

In [11]:
y.add_(x)
print(y)

tensor([[1.0374, 1.3315, 1.5367],
        [0.7699, 2.6684, 1.3390],
        [1.1471, 0.3305, 2.6269],
        [1.5162, 1.4697, 1.6990],
        [1.1867, 2.1853, 1.9199]])


## indexing

In [22]:
print(x[:, 1])

tensor([0.2013, 0.9988, 0.0809, 0.6114, 0.6788])


## view(reshape)

In [24]:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


## numpy 변환(Bridge)
- Torch Tensor를 numpy 배열로 변환하거나 그 반대

In [31]:
a = torch.ones(5)
print(a)

tensor([1., 1., 1., 1., 1.])


In [32]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


In [33]:
a.add_(1)
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


### Numpy 배열을 Torch Tensor로 변환

In [34]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


### CUDA Tensors
- `.cuda()`메소드를 사용하여 Tensor를 GPU상으로 옮길 수 있음

In [42]:
if torch.cuda.is_available():
    x = x.cuda()
    y = y.view((4,4)).cuda()
    print(x+y)

tensor([[ 2.6293,  2.8142, -0.2479, -2.4970],
        [ 0.8423, -0.6348, -1.3439,  0.1074],
        [-0.2339, -0.0628, -0.6163, -2.5177],
        [ 1.1510,  2.1147,  1.0034,  3.0740]], device='cuda:0')


# Autograd : 자동 미분
- Pytorch의 모든 신경망의 중심에는 `autograd`패키지가 있음
- Tensor의 모든 연산에 대해 자동 미분을 제공
    - 실행-기반-정의 프레임워크
    - 역전파는 학습 과정의 매 단계마다 달라짐
    
## 변수(Variable)
- `autograd.Variable`
    - Tensor를 감싸고있으며, Tensor 기반으로 정의된 거의 대부분의 연산을 지원
    - 계산이 완료된 후 `.backward()`를 호출하여 모든 변화도(gradient)를 자동으로 계산
    - `.data`속성을 사용하여 tensor 자체에 접근할 수 있으며, 이 변수와 관련된 변화도는 `.grad`에 누적됨
        - ![image](https://user-images.githubusercontent.com/28910538/63827377-db6e5400-c99d-11e9-8b4d-c93d5803a673.png)

## Fcuntion
- Variable과 Function은 상호 연결되어 있음
- 모든 연산 과정을 부호화(encode)하여 순환하지 않은 그래프를 생성
- 각 변수는 `grad_fn`속성을 갖고있음
    - `Variable`을 생성한 Function을 참조하고 있음
        - 단 사용자가 만든 Variable은 예외(이 때의 grad_fn은 None
- 도함수를 계산하기 위해서는 Variable의 `.backward()`를 호출
- `Variable`이 스칼라인 경우에는 `backward`에 인자를 정해줄 필요가 없음, 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 gradient의 인자로 지정할 필요가 있음

In [43]:
import torch
from torch.autograd import Variable

x = Variable(torch.ones(2,2), requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [44]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [48]:
print(y.grad_fn)

In [49]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


### 변화도(Gradient)
- `out.backward()`는 `out.backward(torch.Tensor([1,0]))`와 같음

In [50]:
out.backward()

In [51]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [53]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-880.0632, -283.4705, -426.4141], grad_fn=<MulBackward0>)


In [54]:
gradient = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradient)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


# 신경망(Neural Networks)
- `torch.nn`패키지 사용해 쉽게 생성 가능
- 모델을 정의하고 미분하는데 `autograd`를 사용
- `nn.Module`은 계층과 `output`을 반환하는 `forward(input)`메서드를 포함하고 있음

## 신경망의 전형적인 학습 과정
- 학습 가능한 매개변수(또는 가중치)를 갖는 신경망을 정의
- 데이터 셋 입력을 반복
- 입력을 신경망에서 처리
- loss(출려과 정답이 얼마나 떨어져있는지)를 계산
- 변화도(gradient)를 신경망의 매개변수들에 역으로 전파
- 신경망의 가중치 갱신(weight = learning_rate * wieght * gradient)

## 신경망 정의

In [56]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # input 채널, output 채널, kerenl
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, X):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward`함수만 정의하고나면 변화도 계산하는 `backward`함수는 `autograd`를 사용하여 자동으로 정의됨, 모델의 학습 가능한 매개변수들은 `net.parameters()`에 의해 반환 됨

In [64]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])
